In [2]:
from scipy.sparse import csr_matrix, lil_matrix, load_npz, save_npz, hstack
import numpy as np 
import pandas as pd 
from tqdm import tqdm 

In [3]:
path = '../../data/Mellody_tuner/all/output_all/pseudolabels_w_melloddy/matrices/cls/cls_T10_y.npz'
y = load_npz(path)

In [4]:
y

<419424x2950 sparse matrix of type '<class 'numpy.float32'>'
	with 3681342 stored elements in Compressed Sparse Row format>

In [5]:
path = './files/baseline_plus_aux_data/pseudolabels_plus_baseline/matrices/cls/cls_T10_y.npz'
y_aux = load_npz(path)

In [6]:
y_aux

<422185x2963 sparse matrix of type '<class 'numpy.float32'>'
	with 4768168 stored elements in Compressed Sparse Row format>

In [7]:
# the aux tasks have the highest index numbers 
y_out = y_aux[:,:y.shape[1]]

In [8]:
y_out

<422185x2950 sparse matrix of type '<class 'numpy.float32'>'
	with 4270810 stored elements in Compressed Sparse Row format>

In [9]:
T10c_cont_aux = pd.read_csv("./files/baseline_plus_aux_data/pseudolabels_plus_baseline/results/T10c_cont.csv")


In [13]:
aux_assays = list(T10c_cont_aux.input_assay_id.unique())

In [ ]:
T10c_cont

In [14]:
T10c_cont_baseline = pd.read_csv("../../data/Mellody_tuner/all/output_all/pseudolabels_w_melloddy/results/T10c_cont.csv")

basline_assays = list(T10c_cont_baseline.input_assay_id.unique())

In [15]:
T10c_cont_baseline[~T10c_cont_baseline['input_assay_id'].isin(T10c_cont_aux.input_assay_id.unique())]['input_assay_id'].drop_duplicates().to_frame()


,input_assay_id
3348431,899993


In [17]:
len(aux_assays), len(basline_assays), len(aux_assays)-len(basline_assays)

(1586, 1558, 28)

In [32]:
df = pd.merge(T10c_cont_baseline[['input_assay_id','cont_classification_task_id']].drop_duplicates(), T10c_cont_aux[['input_assay_id','cont_classification_task_id']].drop_duplicates(), on='cont_classification_task_id', how="outer", indicator=True)
df = df[df['_merge'] == 'both']


In [34]:
df

,input_assay_id_x,cont_classification_task_id,input_assay_id_y,_merge
0,517.0,0,517,both
1,1160.0,1,1160,both
2,1160.0,2,1160,both
3,1512.0,3,1512,both
4,1512.0,4,1512,both
...,...,...,...,...
2945,4000027.0,2945,10000021,both
2946,4000029.0,2946,10000022,both
2947,4000030.0,2947,10000023,both
2948,4000031.0,2948,10000024,both


In [40]:
pd.set_option('display.max_rows', None)

In [41]:
df[~(df['input_assay_id_y'] == df['input_assay_id_x'])]

,input_assay_id_x,cont_classification_task_id,input_assay_id_y,_merge
158,154524.0,158,154975,both
160,154975.0,160,155200,both
162,155200.0,162,155202,both
164,155202.0,164,157209,both
166,157209.0,166,157693,both
168,157693.0,168,157713,both
170,157713.0,170,158033,both
171,158033.0,171,158347,both
173,158347.0,173,158540,both
175,158540.0,175,158753,both


In [42]:
path = './files/baseline_plus_aux_data/pseudolabels_plus_baseline/matrices/cls/cls_T10_y_noaux.npz'
save_npz(path, y_out)

In [43]:
## Redo here for the model training

In [44]:
y_aux_empty = csr_matrix((y_aux.shape[0],y_aux.shape[1] - y_out.shape[1]))

In [45]:
hstack((y_out,y_aux_empty))

<422185x2963 sparse matrix of type '<class 'numpy.float64'>'
	with 4270810 stored elements in COOrdinate format>

In [47]:
path = './files/baseline_plus_aux_data/pseudolabels_plus_baseline/matrices/cls/cls_T10_y_noaux.npz'
save_npz(path,hstack((y_out,y_aux_empty)))

In [ ]:
# Generate the y-sparse matrix for inference 
# Main tasks + fold 0 to limit the amount of predictions

In [48]:
path = './files/baseline_plus_aux_data/pseudolabels_plus_baseline/matrices/cls/cls_T11_fold_vector.npy'
folds = np.load(path)

In [49]:
len(folds)

422185

In [50]:
y_aux

<422185x2963 sparse matrix of type '<class 'numpy.float32'>'
	with 4768168 stored elements in Compressed Sparse Row format>

In [52]:
path = './cp/summary_eps_0.05.csv'
cp = pd.read_csv(path)

In [53]:
col_idxs = list(cp.query('validity_0>0').query('validity_1>0')['index'])

In [54]:
y_sparse_main_fold0 = lil_matrix(y_aux.shape)

In [55]:
y_sparse_main_fold0

<422185x2963 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in List of Lists format>

In [67]:
(folds == 0).sum()

79067

In [57]:
for col in tqdm(col_idxs) :
    y_sparse_main_fold0[folds == 0,col] = 1

100%|██████████| 55/55 [00:00<00:00, 55.20it/s]


In [68]:
y_sparse_main_fold0

<422185x2963 sparse matrix of type '<class 'numpy.float64'>'
	with 4348685 stored elements in List of Lists format>

In [59]:
path = './files/baseline_plus_aux_data/pseudolabels_plus_baseline/matrices/cls/y_sparse_main_fold0.npy'
save_npz(path,csr_matrix(y_sparse_main_fold0))

In [60]:
pwd

'/home/jovyan/Melloddy/aux_data/pseudolabel_auxdata/scripts_and_notebooks'